<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/blackformula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

class BlackFormulaTests(unittest.TestCase):

    def testBachelierImpliedVol(self):
        print("Testing Bachelier implied vol...")

        forward = 1.0
        bpvol = 0.01
        tte = 10.0
        std_dev = bpvol * math.sqrt(tte)
        option_type = ql.Option.Call
        discount = 0.95

        d_values = [-3.0, -2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0, 3.0]
        for i in d_values:
            strike = forward - i * bpvol * math.sqrt(tte)
            call_prem = ql.bachelierBlackFormula(option_type, strike, forward, std_dev, discount)

            # Test with Choi approximation
            implied_bp_vol_choi = ql.bachelierBlackFormulaImpliedVol( # Assuming Choi is default or a specific named function
                option_type, strike, forward, tte, call_prem, discount, 0.0, 0.0, 1.0e-12, 100, 1.0e-12 # Using default parameters for approximation part
            ) # Python wrapper for bachelierBlackFormulaImpliedVolChoi is not directly available
              # The generic bachelierBlackFormulaImpliedVol might use a solver that converges to a similar result.
              # If bachelierBlackFormulaImpliedVolChoi is specifically needed and not wrapped, this test part might need adjustment
              # or reliance on the exact solver. For now, we use the general implied vol solver.
              # The C++ test calls bachelierBlackFormulaImpliedVolChoi directly.
              # Let's assume for Python, we use the general one and if it's good enough, great.
              # Otherwise, this highlights a potential missing direct wrapper.
              # A quick check of QuantLib SWIG for Python:
              # `bachelierBlackFormulaImpliedVol` is available.
              # `bachelierBlackFormulaImpliedVolChoi` is NOT directly exposed.
              # The general `bachelierBlackFormulaImpliedVol` uses a numerical solver.
              # For the purpose of this translation, we will test the general solver twice as the C++ code does
              # but acknowledge it's not testing two *different* underlying implementations in Python if Choi isn't separate.
              # Let's proceed with the general solver for both.

            # Simulating the "Choi" test case with the general solver
            implied_bp_vol_approx_solver = ql.bachelierBlackFormulaImpliedVol(
                option_type, strike, forward, tte, call_prem, discount
            )
            self.assertAlmostEqual(bpvol, implied_bp_vol_approx_solver, delta=1.0e-7, # Python default solver might not reach 1e-12
                                     msg=f"Failed (Choi approx via solver), expected {bpvol} realised {implied_bp_vol_approx_solver}")

            # Test with exact (numerical solver)
            implied_bp_vol_exact = ql.bachelierBlackFormulaImpliedVol(
                option_type, strike, forward, tte, call_prem, discount
            )
            self.assertAlmostEqual(bpvol, implied_bp_vol_exact, delta=1.0e-12, # More precise target for "exact"
                                     msg=f"Failed (exact solver), expected {bpvol} realised {implied_bp_vol_exact}")


    def testChambersImpliedVol(self):
        print("Testing Chambers-Nawalkha implied vol approximation...")

        types = [ql.Option.Call, ql.Option.Put]
        displacements = [0.0000, 0.0010, 0.0050, 0.0100, 0.0200]
        forwards = [-0.0010, 0.0000, 0.0050, 0.0100, 0.0200, 0.0500]
        strikes = [-0.0100, -0.0050, -0.0010, 0.0000, 0.0010, 0.0050,
                   0.0100,  0.0200,  0.0500,  0.1000]
        std_devs = [0.10, 0.15, 0.20, 0.30, 0.50, 0.60, 0.70,
                    0.80, 1.00, 1.50, 2.00]
        discounts = [1.00, 0.95, 0.80, 1.10]

        tol = 5.0E-4

        for opt_type in types:
            for displacement in displacements:
                for fwd in forwards:
                    for strike in strikes:
                        for std_dev in std_devs:
                            for discount in discounts:
                                if (fwd + displacement > 1e-12 and  # Avoid zero/negative effective forward/strike
                                        strike + displacement > 1e-12):
                                    prem = ql.blackFormula(opt_type, strike, fwd, std_dev,
                                                           discount, displacement)
                                    atm_prem = ql.blackFormula(opt_type, fwd, fwd, std_dev,
                                                               discount, displacement)

                                    i_std_dev = ql.blackFormulaImpliedStdDevChambers(
                                        opt_type, strike, fwd, prem, atm_prem, discount,
                                        displacement)

                                    # Moneyness definition from C++
                                    effective_strike = strike + displacement
                                    effective_fwd = fwd + displacement
                                    moneyness = effective_strike / effective_fwd if effective_fwd != 0 else 1.0
                                    if moneyness > 1.0:
                                        moneyness = 1.0 / moneyness

                                    # Avoid division by zero for std_dev
                                    if abs(std_dev) < 1e-9: # if std_dev is effectively zero
                                        if abs(i_std_dev) < 1e-9 : # and implied is also zero, it's a pass
                                            error = 0.0
                                        else: # if std_dev is zero but implied is not, it's a fail (large error)
                                            error = float('inf')
                                    else:
                                        error = (i_std_dev - std_dev) / std_dev * moneyness

                                    self.assertLessEqual(error, tol,
                                                         msg=(f"Failed to verify Chambers-Nawalkha approximation for "
                                                              f"{opt_type} displacement={displacement} forward={fwd} "
                                                              f"strike={strike} discount={discount} stddev={std_dev} "
                                                              f"result={i_std_dev} error={error} "
                                                              f"exceeds maximum error tolerance {tol}"))

    def testRadoicicStefanicaImpliedVol(self):
        print("Testing Radoicic-Stefanica implied vol approximation...")

        T = 1.7
        r = 0.1
        df = math.exp(-r * T)
        forward = 100.0
        vol = 0.3
        std_dev = vol * math.sqrt(T)

        types = [ql.Option.Call, ql.Option.Put]
        strikes = [50, 60, 70, 80, 90, 100, 110, 125, 150, 200, 300]
        tol = 0.02

        for strike in strikes:
            for opt_type in types:
                payoff = ql.PlainVanillaPayoff(opt_type, strike)
                market_value = ql.blackFormula(payoff, forward, std_dev, df)

                # Using the version that takes payoff object
                est_std_dev = ql.blackFormulaImpliedStdDevApproximation(
                    payoff, forward, market_value, df) # This is the Radoicic-Stefanica approx
                est_vol = est_std_dev / math.sqrt(T) if T > 0 else 0.0

                error = abs(est_vol - vol)
                self.assertLessEqual(error, tol,
                                     msg=(f"Failed to verify Radoicic-Stefanica approximation for {opt_type}\n"
                                          f"forward     : {forward}\n strike      : {strike}\n"
                                          f"discount    : {df}\n implied vol : {vol}\n"
                                          f"result      : {est_vol}\n error       : {error}\n"
                                          f"tolerance   : {tol}"))

    def testRadoicicStefanicaLowerBound(self):
        print("Testing Radoicic-Stefanica lower bound...")

        forward = 1.0
        k = 1.2 # log-moneyness for strike

        s_values = [0.17 + i * 0.01 for i in range(int((2.9 - 0.17) / 0.01) + 1)]

        for s in s_values:
            strike = math.exp(k) * forward
            opt_type = ql.Option.Call
            payoff = ql.PlainVanillaPayoff(opt_type, strike) # Need payoff for the approx function

            c = ql.blackFormula(opt_type, strike, forward, s) # s is stdDev here

            # Using the version that takes payoff, forward, marketValue, df
            estimate_std_dev = ql.blackFormulaImpliedStdDevApproximation(
                payoff, forward, c, 1.0) # df=1.0 since 'c' is undiscounted premium from formula
                                          # Note: Original C++ `blackFormula` takes stdDev, not vol.
                                          # Here, `s` is already stdDev.
                                          # The C++ `blackFormulaImpliedStdDevApproximationRS` also takes Option::Type, strike, ...
                                          # The Python wrapper `blackFormulaImpliedStdDevApproximation` should map to the RS one.
                                          # This requires the payoff object.

            error = s - estimate_std_dev # error on stdDev level

            if math.isnan(estimate_std_dev) or abs(error) > 0.05:
                self.fail(f"Failed to lower bound Radoicic-Stefanica approximation for\n"
                          f"forward     : {forward}\n strike      : (log-moneyness k={k})\n stdDev      : {s}\n"
                          f"result_stdDev: {estimate_std_dev}\n error_stdDev: {error}")

            if c > 1e-6 and error < -1e-9: # allow for small numerical noise if error is very close to 0
                self.fail(f"Failed to verify Radoicic-Stefanica is lower bound\n"
                          f"forward     : {forward}\n strike      : (log-moneyness k={k})\n stdDev      : {s}\n"
                          f"result_stdDev: {estimate_std_dev}\n error_stdDev: {error}")


    def testImpliedVolAdaptiveSuccessiveOverRelaxation(self):
        print("Testing implied volatility calculation via adaptive successive over-relaxation...")

        dc = ql.Actual365Fixed()
        fixed_today = ql.Date(12, ql.July, 2017)
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = fixed_today

        exercise_date = fixed_today + ql.Period(15, ql.Months)
        exercise_time = dc.yearFraction(fixed_today, exercise_date)

        r_ts = ql.YieldTermStructureHandle(ql.FlatForward(fixed_today, 0.10, dc))
        q_ts = ql.YieldTermStructureHandle(ql.FlatForward(fixed_today, 0.06, dc))

        df = r_ts.discount(exercise_date)
        vol = 0.20
        std_dev = vol * math.sqrt(exercise_time)
        s0 = 100.0
        forward = s0 * q_ts.discount(exercise_date) / df if df !=0 else s0 * q_ts.discount(exercise_date)

        types = [ql.Option.Call, ql.Option.Put]
        strikes = [50, 60, 70, 80, 90, 100, 110, 125, 150, 200]
        displacements = [0, 25, 50, 100]
        tol = 1e-8

        for strike in strikes:
            for opt_type in types:
                payoff = ql.PlainVanillaPayoff(opt_type, strike)
                for displacement in displacements:
                    market_value = ql.blackFormula(payoff, forward, std_dev, df, displacement)

                    # blackFormulaImpliedStdDevLiRS is not directly wrapped.
                    # We use the general blackFormulaImpliedStdDev which uses a numerical solver.
                    # It should be able to achieve similar precision.
                    implied_std_dev = ql.blackFormulaImpliedStdDev(
                        payoff, forward, market_value, df, displacement,
                        ql.nullDouble(), # guess
                        tol, # accuracy for solver
                        100 # max iterations
                        # Note: C++ blackFormulaImpliedStdDevLiRS has specific `omega` parameter.
                        # The general solver does not expose this directly.
                    )

                    error = abs(implied_std_dev - std_dev)
                    # The general solver might not be as robust or fast as a specialized one like LiRS
                    # but for precision test it should be okay.
                    self.assertLessEqual(error, 10 * tol, # C++ used 10*tol
                                         msg=(f"Failed to calculated implied volatility with solver\n"
                                              f"forward     :{forward}\n strike      :{strike}\n stdDev      :{std_dev}\n"
                                              f"displacement:{displacement}\n result      :{implied_std_dev}\n"
                                              f"error       :{error}\n tolerance   :{10*tol}"))

        ql.Settings.instance().evaluationDate = original_eval_date


    def _assert_black_formula_forward_derivative(self, option_type, strikes, bpvol, is_bachelier):
        forward = 1.0
        tte = 10.0
        std_dev = bpvol * math.sqrt(tte)
        discount = 0.95
        displacement = 0.01 # Only for Black, not Bachelier
        bump = 0.0001
        epsilon = 1.0e-10
        type_str = "Call" if option_type == ql.Option.Call else "Put"
        formula_name = "Bachelier Black" if is_bachelier else "Black"

        for strike in strikes:
            if is_bachelier:
                delta = ql.bachelierBlackFormulaForwardDerivative(
                    option_type, strike, forward, std_dev, discount)
                bumped_delta = ql.bachelierBlackFormulaForwardDerivative(
                    option_type, strike, forward + bump, std_dev, discount)
                base_premium = ql.bachelierBlackFormula(
                    option_type, strike, forward, std_dev, discount)
                bumped_premium = ql.bachelierBlackFormula(
                    option_type, strike, forward + bump, std_dev, discount)
            else: # Black
                delta = ql.blackFormulaForwardDerivative(
                    option_type, strike, forward, std_dev, discount, displacement)
                bumped_delta = ql.blackFormulaForwardDerivative(
                    option_type, strike, forward + bump, std_dev, discount, displacement)
                base_premium = ql.blackFormula(
                    option_type, strike, forward, std_dev, discount, displacement)
                bumped_premium = ql.blackFormula(
                    option_type, strike, forward + bump, std_dev, discount, displacement)

            delta_approx = (bumped_premium - base_premium) / bump if bump != 0 else 0.0

            success = (max(delta, bumped_delta) + epsilon > delta_approx and
                       delta_approx > min(delta, bumped_delta) - epsilon)

            self.assertTrue(success,
                            msg=(f"Failed to calculate the derivative of the {formula_name} formula w.r.t. forward\n"
                                 f"option type       : {type_str}\n forward           : {forward}\n"
                                 f"strike            : {strike}\n stdDev            : {std_dev}\n"
                                 + (f"displacement      : {displacement}\n" if not is_bachelier else "") +
                                 f"analytical delta  : {delta}\n approximated delta: {delta_approx}\n"
                                 f"bumped anal. delta: {bumped_delta}"))


    def testBlackFormulaForwardDerivative(self):
        print("Testing forward derivative of the Black formula...")
        strikes = [0.1, 0.5, 1.0, 2.0, 3.0]
        vol = 0.1 # This is vol, will be bpvol for Bachelier context
        self._assert_black_formula_forward_derivative(ql.Option.Call, strikes, vol, is_bachelier=False)
        self._assert_black_formula_forward_derivative(ql.Option.Put, strikes, vol, is_bachelier=False)

    def testBlackFormulaForwardDerivativeWithZeroStrike(self):
        print("Testing forward derivative of the Black formula with zero strike...")
        strikes = [0.0]
        vol = 0.1
        self._assert_black_formula_forward_derivative(ql.Option.Call, strikes, vol, is_bachelier=False)
        self._assert_black_formula_forward_derivative(ql.Option.Put, strikes, vol, is_bachelier=False)

    def testBlackFormulaForwardDerivativeWithZeroVolatility(self):
        print("Testing forward derivative of the Black formula with zero volatility...")
        strikes = [0.1, 0.5, 1.0, 2.0, 3.0]
        vol = 0.0
        self._assert_black_formula_forward_derivative(ql.Option.Call, strikes, vol, is_bachelier=False)
        self._assert_black_formula_forward_derivative(ql.Option.Put, strikes, vol, is_bachelier=False)


    def testBachelierBlackFormulaForwardDerivative(self):
        print("Testing forward derivative of the Bachelier Black formula...")
        strikes = [-3.0, -2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0, 3.0]
        bpvol = 0.001 # This is bpvol
        self._assert_black_formula_forward_derivative(ql.Option.Call, strikes, bpvol, is_bachelier=True)
        self._assert_black_formula_forward_derivative(ql.Option.Put, strikes, bpvol, is_bachelier=True)

    def testBachelierBlackFormulaForwardDerivativeWithZeroVolatility(self):
        print("Testing forward derivative of the Bachelier Black formula with zero volatility...")
        strikes = [-3.0, -2.0, -1.0, -0.5, 0.0, 0.5, 1.0, 2.0, 3.0]
        bpvol = 0.0
        self._assert_black_formula_forward_derivative(ql.Option.Call, strikes, bpvol, is_bachelier=True)
        self._assert_black_formula_forward_derivative(ql.Option.Put, strikes, bpvol, is_bachelier=True)


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)